In [ ]:
# !pip install imageio

In [ ]:
# !python train.py -h

In [ ]:
# files modified
"""
Pluralistic/train.py
Pluralistic/options/base_options.py
Pluralistic/dataloader/data_loader.py
"""
# --continue_train

In [5]:
from object_detect.generate_masks import create_rand_mask, create_mask_for_object

In [ ]:
img_dir = 'linmao-test_txt/JPEGImages'
label_dir = 'linmao-test_txt/Annotations'
save_dir = 'linmao-test_txt/Masks'
save_txt_dir = 'linmao-test_txt/Masks_txt'
create_rand_mask(img_dir, label_dir, save_dir, save_txt_dir, num_per_img=4,
                     width_minmax=(0.05, 0.25),
                     hight_minmax=(0.05, 0.25),
                     img_type='jpg',
                     save_type='jpg')

In [12]:
save_obj_mask_dir = 'linmao-test_txt/ObjMasks'
create_mask_for_object(img_dir, label_dir, save_obj_mask_dir, expand_ratio_width=0, expand_ratio_hight=0, img_type='jpg',
                save_type='jpg')

100%|██████████| 1047/1047 [00:36<00:00, 28.47it/s]


In [ ]:
from object_detect.box_image_text import plot_box

In [ ]:
import os
out_put_dir = 'linmao-test_txt/Masks_on_origin'
os.makedirs(out_put_dir, exist_ok=True)
plot_box(img_dir, save_txt_dir, out_put_dir, thickness=20, filter_diff=True)

# Note
### Put mask_type as 3 and mask_file in order to use my own masks, the image name and mask name should match

In [ ]:
!python train.py --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/Masks --niter 100000 --save_iters_freq 100000 --save_latest_freq 1 --batchSize 4 --mask_type 3 --continue_train



--------------Options--------------
batchSize: 4
checkpoints_dir: ./checkpoints
continue_train: True
display_freq: 100
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: [512, 512]
gan_mode: lsgan
gpu_ids: 0
img_file: linmao-test_txt/JPEGImages
isTrain: True
iter_count: 1
lambda_g: 1.0
lambda_kl: 20.0
lambda_rec: 20.0
loadSize: [522, 522]
lr: 0.0001
lr_policy: lambda
mask_file: linmao-test_txt/Masks
mask_type: 3
model: pluralistic
nThreads: 8
name: linmao
niter: 100000
niter_decay: 0
no_augment: False
no_flip: False
no_html: False
no_rotation: False
no_shuffle: False
output_scale: 4
print_freq: 100
resize_or_crop: resize_and_crop
save_iters_freq: 100000
save_latest_freq: 1
train_paths: two
which_iter: latest
----------------End----------------
training images = 1048
ResEncoder(
  (block0): ResBlockEncoderOptimized(
    (conv1): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): Spec

In [ ]:
# !python test.py -h

# batchSize must be 1 in order to get correct predictions
# Modify Pluralistic/options/test_options.py how_many to set up how many pictures to predict

In [4]:
!python test.py  --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/Masks --mask_type 3 --batchSize 1


--------------Options--------------
batchSize: 1
checkpoints_dir: ./checkpoints
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: [512, 512]
gpu_ids: 0
how_many: 10
img_file: linmao-test_txt/JPEGImages
isTrain: False
loadSize: [522, 522]
mask_file: linmao-test_txt/Masks
mask_type: 3
model: pluralistic
nThreads: 8
name: linmao
no_augment: False
no_flip: False
no_rotation: False
no_shuffle: False
nsampling: 1
ntest: inf
output_scale: 4
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
save_number: 1
which_iter: latest
----------------End----------------
testing images = 1047
ResEncoder(
  (block0): ResBlockEncoderOptimized(
    (conv1): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): SpectralNorm(
      (module): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (bypass): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(1, 1), st

In [13]:
!python test.py  --name linmao --img_file linmao-test_txt/JPEGImages --mask_file linmao-test_txt/ObjMasks --mask_type 3 --batchSize 1


--------------Options--------------
batchSize: 1
checkpoints_dir: ./checkpoints
display_id: 1
display_port: 8097
display_single_pane_ncols: 0
display_winsize: 256
fineSize: [512, 512]
gpu_ids: 0
how_many: 10
img_file: linmao-test_txt/JPEGImages
isTrain: False
loadSize: [522, 522]
mask_file: linmao-test_txt/ObjMasks
mask_type: 3
model: pluralistic
nThreads: 8
name: linmao
no_augment: False
no_flip: False
no_rotation: False
no_shuffle: False
nsampling: 1
ntest: inf
output_scale: 4
phase: test
resize_or_crop: resize_and_crop
results_dir: ./results/
save_number: 1
which_iter: latest
----------------End----------------
testing images = 1047
ResEncoder(
  (block0): ResBlockEncoderOptimized(
    (conv1): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (conv2): SpectralNorm(
      (module): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (bypass): SpectralNorm(
      (module): Conv2d(3, 32, kernel_size=(1, 1),

In [ ]:
import torchvision.transforms as transforms
from PIL import Image, ImageFile

In [ ]:
loadSize = [522, 522]
fineSize = [512, 512]

In [ ]:
transform_list = []
osize = [loadSize[0], loadSize[1]]
fsize = [fineSize[0], fineSize[1]]

transform_list.append(transforms.Resize(fsize))

transform_list += [transforms.ToTensor()]
transform = transforms.Compose(transform_list)

In [ ]:
img_path = 'linmao-test_txt/JPEGImages/nl_0438_0420.jpg'
img_pil = Image.open(img_path).convert('RGB')
img = transform(img_pil)

In [ ]:
import numpy as np
np.array(img_pil).shape

In [ ]:
img.shape

In [ ]:
import cv2


image = img.numpy()
image = cv2.cvtColor(image)

pixels = np.array(image)

plt.imshow(pixels)
plt.show()

In [ ]:
from matplotlib import pyplot as plt
img = img.numpy()
# img = img.reshape(img.shape[1], img.shape[2], img.shape[0])
from matplotlib import pyplot as plt
plt.imshow(img, interpolation='nearest')